## **4. Keywords Clustering** 
We will compare different models implemeting each of these parameters:
- **Algorithm**: K-Means vs Expectation maximization VS Agglomerative
- **Embedding** : One-hot vs Sentence transformers
- **Number of clusters** : ranging from N/10, N/2 the total number of terms)

*Dernières modifications à ajouter le lundi 15 août* :  
- Ajouter les to_csv pour pouvoir regarder les résultats (pour chaque algorithme, pour chaque embedding, avec les meilleurs paramètres pour la dimensionnalité et le nombre de clusters)
- Modifier la façon de réduire les dimensions pour les one-hot embeddings pour que ce soit aussi une PCA (ce sera plus rigoureux si on procède de la même manière pour les deux types d'embeddings et c'est cette méthode qu'on utilise pour les sentence transformers)
- Ajouter d'autres critères de performance que le silhouette score : https://datascience.oneoffcoder.com/kmc-bic-aic.html
- Extraire des graphiques / courbes (est-ce une bonne idée ?)
- Clusteriser les formes lemmatisées (?)

In [249]:
acteur = 'chum'

In [250]:
import glob
from pandas import *
from collections import Counter
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [251]:
# # get data file names
# path ='../05-transformation'
# filenames = glob.glob(path + "/*.csv")

# dfs = []
# for filename in filenames:
#     dfs.append(read_csv(filename))

# # Concatenate all data into one DataFrame
# df = concat(dfs, ignore_index=True)

In [252]:
base_path ='../05-transformation/'
file_path = base_path + acteur + '_weighting_OKapiBM25.csv'

with open(file_path, encoding='utf-8') as f:
    df = read_csv(f).drop(columns=["Unnamed: 0", 'Structure syntaxique', 'LLR', 'TF (sklearn)', 'DF (sklearn)', 'TF-IDF', 'OkapiBM25', 'Terme formatté'])
df['Fréquence totale (TF)'] = df.groupby(['Terme'])['Fréquence (TF)'].transform('sum')
df['Fréquence documentaire totale (DF)'] = df.groupby(['Terme'])['Fréquence documentaire (DF)'].transform('sum')
df = df.drop(columns = ['Corpus', 'Fréquence (TF)', 'Fréquence documentaire (DF)'])
df = df.drop_duplicates(subset=['Terme'])
df['Terme'] = df['Terme'].astype('str')
df['TF + DF'] = df['Fréquence totale (TF)'] + df['Fréquence documentaire totale (DF)']

df

,Terme,isMeSHTerm,isTaxoTerm,Fréquence totale (TF),Fréquence documentaire totale (DF),TF + DF
0,chirurgiens du canada,False,False,46,46,92
1,réunions hebdomadaires,False,False,86,86,172
2,centre hospitalier de l'université,False,False,115,88,203
3,activité de développement professionnel,False,False,43,43,86
4,centre de recherche du centre,False,False,78,68,146
...,...,...,...,...,...,...
179,professeur au département,False,False,34,26,60
180,chercheurs du crchum,False,False,37,30,67
181,recherche chirurgie,False,False,33,32,65
182,calendrier des conférences,False,False,40,40,80


In [253]:
algorithmes = ['K-Means', 'Expectation-Maximization', 'AgglomerativeClustering']
embeddings = ['One-Hot', 'Sentence transformers']

nb_termes = len(df['Terme'].tolist())
clusters = range(round(nb_termes/10), round(nb_termes/2))

results = []
for algorithme in algorithmes:
    for embedding in embeddings:
        for cluster in clusters:
            results.append(\
            {'algorithme' : algorithme,\
                'embedding': embedding, \
                'K (nb clusters)' : cluster, \
                'Score Silhouette': None})


# On va remplir ce dictionnaire avec les bons scores au fur et à mesure qu'on expérimente
results = DataFrame(results)
results

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,None
1,K-Means,One-Hot,19,None
2,K-Means,One-Hot,20,None
3,K-Means,One-Hot,21,None
4,K-Means,One-Hot,22,None
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,None
440,AgglomerativeClustering,Sentence transformers,88,None
441,AgglomerativeClustering,Sentence transformers,89,None
442,AgglomerativeClustering,Sentence transformers,90,None


In [254]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")

file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]

def to_tokens(kw, min_chars=2):
    tokens = tokenizer.tokenize(str(kw)) # split the string into a list of words
    tokens = [word for word in tokens if len(word) > min_chars] 
    tokens = [str(word) for word in tokens if word not in stopwords] 
    
    tokens = set(tokens) # to remove duplicates
    tokens = sorted(tokens) # converts our set back to a list and sorts words in alphabetical order
    return tokens

In [255]:
df['tokens'] = df["Terme"].apply(lambda x: to_tokens(
    x,
    min_chars=2,
))


# # Test - Seulement retenir des n-grammes où n est au-dessus de 2 - en fait pas besoin
# df["len"] = df["tokens"].apply(lambda x : len(x))
# df = df[df['len'] > 1].drop(columns=["len"])

df

,Terme,isMeSHTerm,isTaxoTerm,Fréquence totale (TF),Fréquence documentaire totale (DF),TF + DF,tokens
0,chirurgiens du canada,False,False,46,46,92,"[canada, chirurgiens]"
1,réunions hebdomadaires,False,False,86,86,172,"[hebdomadaires, réunions]"
2,centre hospitalier de l'université,False,False,115,88,203,"[centre, hospitalier, université]"
3,activité de développement professionnel,False,False,43,43,86,"[activité, développement, professionnel]"
4,centre de recherche du centre,False,False,78,68,146,"[centre, recherche]"
...,...,...,...,...,...,...,...
179,professeur au département,False,False,34,26,60,"[département, professeur]"
180,chercheurs du crchum,False,False,37,30,67,"[chercheurs, crchum]"
181,recherche chirurgie,False,False,33,32,65,"[chirurgie, recherche]"
182,calendrier des conférences,False,False,40,40,80,"[calendrier, conférences]"


In [256]:
vocab = sorted(set(df["tokens"].explode()))
len(vocab)

dim = len(vocab)
dim

158

In [293]:
vocab

['département',
 'radiologie',
 'axe',
 'imagerie',
 'santé',
 'évaluation',
 'fondamentale',
 'recherche',
 'mcgill',
 'université',
 'animaux',
 'modèles',
 'titulaire',
 'immunitaire',
 'système',
 'chercheur',
 'frcsc',
 'travaux',
 'maladie',
 'parkinson',
 'service',
 'urgence',
 'neurosciences',
 'agrégée',
 'professeure',
 'chum',
 'crchum',
 'professeur',
 'québec',
 'publique',
 'demande',
 'équipes',
 'canada',
 'erik',
 'joly',
 'amphithéâtre',
 'hebdomadaires',
 'réunions',
 'magnétique',
 'résonance',
 'cardiaque',
 'insuffisance',
 'psychiatrie',
 'centre',
 'chercheurs',
 'projet',
 'anne-marie',
 'mes-masson',
 'plaques',
 'sclérose',
 'cardiométabolique',
 'canadienne',
 'société',
 'qualité',
 'vie',
 'cliniques',
 'essais',
 'pubmed',
 'irsc',
 'clinique',
 'agrégé',
 'projets',
 'mentale',
 'médecine',
 'nucléaire',
 'conférenciers',
 'renom',
 'scientifiques',
 'chirurgie',
 'cancer',
 'adjoint',
 'professionnel',
 'appropriées',
 'mesures',
 'immunopathologie',
 

In [257]:
def to_vector(keyword,vocab):
    """
    Calculates vector of keyword on given vocabulary.

    Returns vector as a list of values.  
    """
    vector = []
    for word in vocab:
        if word in keyword:
            vector.append(1)
        else:
            vector.append(0)
    return vector

## **Kmeans**

### One-Hot embedding

> One Hot encoding is a representation of categorical variables as binary vectors. Each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1.

In [292]:
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from sklearn.metrics import calinski_harabasz_score


algo = 'K-Means'
embed = 'One-Hot'


counter = Counter(df["tokens"].explode().to_list())
vocab = []
for key in counter:
    vocab.append(key)

df["vector"] = df["tokens"].apply(lambda x: to_vector(x,vocab))
X = df['vector'].tolist()

df

,Terme,isMeSHTerm,isTaxoTerm,Fréquence totale (TF),Fréquence documentaire totale (DF),TF + DF,tokens,vector,kmeans,clusters
16,département de radiologie,False,True,101,87,188,"[département, radiologie]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",10,1
40,axe imagerie,False,False,37,26,63,"[axe, imagerie]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,1
176,évaluation en santé,False,False,284,271,555,"[santé, évaluation]","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11,1
149,recherche fondamentale,False,False,53,44,97,"[fondamentale, recherche]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",5,1
109,université mcgill,False,False,71,53,124,"[mcgill, université]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...",7,31
...,...,...,...,...,...,...,...,...,...,...
15,chum directeur,False,False,35,43,78,"[chum, directeur]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9,23
156,axe de recherche,False,False,961,959,1920,"[axe, recherche]","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0,23
89,recherche cancer,False,False,203,180,383,"[cancer, recherche]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",1,26
38,assistance immédiate,False,False,912,912,1824,"[assistance, immédiate]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,26


In [ ]:
# Run NMF
# we redo the normalization to improve the k-means result.
nmf = NMF(n_components= round(len(vocab)/5))
normalizer = Normalizer(copy=False)
pip = make_pipeline(nmf, normalizer)

X = pip.fit_transform(X)
for k in clusters:
    kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
    labels = kmeans.labels_

    score  =  silhouette_score(X, labels)
    results.loc[((results['algorithme'] == algo) & \
                (results['embedding'] == embed) & \
                (results['K (nb clusters)'] == k)), 'Score Silhouette'] = score


results

*Stock resulting clusters into a CSV file*

In [259]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))]
                  
# Essayer d'extraire des courbes / graphiques à partir de ce tableau
tab

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
69,K-Means,One-Hot,87,0.607381
70,K-Means,One-Hot,88,0.604163
71,K-Means,One-Hot,89,0.598327
72,K-Means,One-Hot,90,0.596093


In [260]:
params = tab[((tab['algorithme'] == algo) & \
                  (tab['embedding'] == embed) & \
                 (tab['Score Silhouette'] == tab['Score Silhouette'].max()))]

params

,algorithme,embedding,K (nb clusters),Score Silhouette
50,K-Means,One-Hot,68,0.631805


In [261]:
k = params['K (nb clusters)'].values[0]
k

68

In [262]:
kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
labels = kmeans.labels_
df["kmeans"] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(kmeans.labels_.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["kmeans"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['kmeans'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

### Sentence transformers embedding

> "A **transformer** is a deep learning model that adopts the mechanism of self-attention, differentially weighting the significance of each part  of the input data.
Transformers are increasingly the model of choice for NLP problems, replacing RNN models such as long short-term memory (LSTM). The additional  training parallelization allows training on larger datasets. This led to the development of pretrained systems such as BERT (Bidirectional Encoder Representations from Transformers) and GPT (Generative Pre-trained Transformer), which were trained with large language datasets, such as the Wikipedia Corpus and Common Crawl, and can be fine-tuned for specific tasks."   
  
(https://en.wikipedia.org/wiki/Transformer_(machine_learning_model))


In [263]:
from sentence_transformers import SentenceTransformer, models
import torch

# On va utiliser un modèle BERT/sentence transformers (fr) pour extraire nos embeddings plutôt que des simples one-hot encoding
model =  SentenceTransformer("dangvantuan/sentence-camembert-base")

sentences = df['Terme'].tolist()
embeddings_st = model.encode(sentences, convert_to_numpy=True)

In [265]:
algo = 'K-Means'
embed = 'Sentence transformers'

X = embeddings_st

for k in clusters:
    kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
    labels = kmeans.labels_

    score  =  silhouette_score(X, labels)
    results.loc[((results['algorithme'] == algo) & \
                (results['K (nb clusters)'] == k) & \
                (results['embedding'] == embed)), 'Score Silhouette'] = score


results
    

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,None
440,AgglomerativeClustering,Sentence transformers,88,None
441,AgglomerativeClustering,Sentence transformers,89,None
442,AgglomerativeClustering,Sentence transformers,90,None


*Stock resulting clusters into a CSV file*

In [266]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))]
                  
# Essayer d'extraire des courbes / graphiques à partir de ce tableau
tab

,algorithme,embedding,K (nb clusters),Score Silhouette
74,K-Means,Sentence transformers,18,0.15544
75,K-Means,Sentence transformers,19,0.165302
76,K-Means,Sentence transformers,20,0.167321
77,K-Means,Sentence transformers,21,0.177563
78,K-Means,Sentence transformers,22,0.180725
...,...,...,...,...
143,K-Means,Sentence transformers,87,0.251608
144,K-Means,Sentence transformers,88,0.25286
145,K-Means,Sentence transformers,89,0.253773
146,K-Means,Sentence transformers,90,0.257105


In [267]:
params = tab[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed) & \
                 (tab['Score Silhouette'] == tab['Score Silhouette'].max()))]

params

C:\Users\p1115145\AppData\Local\Temp\ipykernel_3084\3434969749.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  params = tab[((results['algorithme'] == algo) & \


,algorithme,embedding,K (nb clusters),Score Silhouette
147,K-Means,Sentence transformers,91,0.26176


In [268]:
k = params['K (nb clusters)'].values[0]
kmeans = KMeans(n_clusters=k, init='k-means++', algorithm='elkan', random_state=0, n_init=1, max_iter=200).fit(X)
labels = kmeans.labels_
df["kmeans"] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(kmeans.labels_.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["kmeans"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['kmeans'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

## **Expectation-Maximization**

In [269]:
from sklearn.mixture import GaussianMixture

### One-Hot embedding

In [270]:
algo = 'Expectation-Maximization'
embed = 'One-Hot'


X = df['vector'].tolist()
for k in clusters:
    gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
    labels = gmm.predict(X)

    score  =  silhouette_score(X, labels)
    results.loc[((results['algorithme'] == algo) & \
                (results['K (nb clusters)'] == k) & \
                (results['embedding'] == embed)), 'Score Silhouette'] = score

results

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,None
440,AgglomerativeClustering,Sentence transformers,88,None
441,AgglomerativeClustering,Sentence transformers,89,None
442,AgglomerativeClustering,Sentence transformers,90,None


*Stock resulting clusters into a CSV file*

In [271]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))]
                  
# Essayer d'extraire des courbes / graphiques à partir de ce tableau
tab

,algorithme,embedding,K (nb clusters),Score Silhouette
148,Expectation-Maximization,One-Hot,18,0.085318
149,Expectation-Maximization,One-Hot,19,0.074609
150,Expectation-Maximization,One-Hot,20,0.079143
151,Expectation-Maximization,One-Hot,21,0.063015
152,Expectation-Maximization,One-Hot,22,0.073585
...,...,...,...,...
217,Expectation-Maximization,One-Hot,87,0.139886
218,Expectation-Maximization,One-Hot,88,0.118419
219,Expectation-Maximization,One-Hot,89,0.104993
220,Expectation-Maximization,One-Hot,90,0.143507


In [272]:
params = tab[(tab['Score Silhouette'] == tab['Score Silhouette'].max())]

params

,algorithme,embedding,K (nb clusters),Score Silhouette
195,Expectation-Maximization,One-Hot,65,0.156311


In [273]:
k = params['K (nb clusters)'].values[0]
gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
labels = gmm.predict(X)
df["clusters"] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(kmeans.labels_.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["clusters"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['clusters'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

### Sentence transformers embedding

In [274]:
algo = 'Expectation-Maximization'
embed = 'Sentence transformers'

X = embeddings_st

for k in clusters:
    gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
    labels = gmm.predict(X)

    score  =  silhouette_score(X, labels)
    results.loc[((results['algorithme'] == 'Expectation-Maximization') & \
                (results['K (nb clusters)'] == k) & \
                (results['embedding'] == 'Sentence transformers')), 'Score Silhouette'] = score

results
    

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,None
440,AgglomerativeClustering,Sentence transformers,88,None
441,AgglomerativeClustering,Sentence transformers,89,None
442,AgglomerativeClustering,Sentence transformers,90,None


*Stock resulting clusters into a CSV file*

In [275]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))]
                  
# Essayer d'extraire des courbes / graphiques à partir de ce tableau
tab

,algorithme,embedding,K (nb clusters),Score Silhouette
222,Expectation-Maximization,Sentence transformers,18,0.148081
223,Expectation-Maximization,Sentence transformers,19,0.138617
224,Expectation-Maximization,Sentence transformers,20,0.142843
225,Expectation-Maximization,Sentence transformers,21,0.162941
226,Expectation-Maximization,Sentence transformers,22,0.16109
...,...,...,...,...
291,Expectation-Maximization,Sentence transformers,87,0.252432
292,Expectation-Maximization,Sentence transformers,88,0.253908
293,Expectation-Maximization,Sentence transformers,89,0.244674
294,Expectation-Maximization,Sentence transformers,90,0.235876


In [276]:
params = tab[(tab['Score Silhouette'] == tab['Score Silhouette'].max())]

params

,algorithme,embedding,K (nb clusters),Score Silhouette
290,Expectation-Maximization,Sentence transformers,86,0.272454


In [277]:
k = params['K (nb clusters)'].values[0]
gmm = GaussianMixture(n_components=k, init_params='k-means++', covariance_type='diag').fit(X) # diag pour gérer MemoryError
labels = gmm.predict(X)
df["clusters"] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(kmeans.labels_.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["clusters"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['clusters'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

## **Agglomerative Clustering**

### One-Hot embedding

In [278]:
from sklearn.cluster import AgglomerativeClustering

In [279]:
algo = 'AgglomerativeClustering'
embed = 'One-Hot'

for k in clusters:
    model = AgglomerativeClustering().fit(X)
    labels = model.labels_

    score  =  silhouette_score(X, labels)
    results.loc[((results['algorithme'] == 'AgglomerativeClustering') &
                (results['K (nb clusters)'] == k) & \
                (results['embedding'] == 'One-Hot')), 'Score Silhouette'] = score

results

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,None
440,AgglomerativeClustering,Sentence transformers,88,None
441,AgglomerativeClustering,Sentence transformers,89,None
442,AgglomerativeClustering,Sentence transformers,90,None


*Stock resulting clusters into a CSV file*

In [280]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))]
                  
# Essayer d'extraire des courbes / graphiques à partir de ce tableau
tab

,algorithme,embedding,K (nb clusters),Score Silhouette
296,AgglomerativeClustering,One-Hot,18,0.027938
297,AgglomerativeClustering,One-Hot,19,0.027938
298,AgglomerativeClustering,One-Hot,20,0.027938
299,AgglomerativeClustering,One-Hot,21,0.027938
300,AgglomerativeClustering,One-Hot,22,0.027938
...,...,...,...,...
365,AgglomerativeClustering,One-Hot,87,0.027938
366,AgglomerativeClustering,One-Hot,88,0.027938
367,AgglomerativeClustering,One-Hot,89,0.027938
368,AgglomerativeClustering,One-Hot,90,0.027938


In [281]:
params = tab[(tab['Score Silhouette'] == tab['Score Silhouette'].max())]

params

,algorithme,embedding,K (nb clusters),Score Silhouette
296,AgglomerativeClustering,One-Hot,18,0.027938
297,AgglomerativeClustering,One-Hot,19,0.027938
298,AgglomerativeClustering,One-Hot,20,0.027938
299,AgglomerativeClustering,One-Hot,21,0.027938
300,AgglomerativeClustering,One-Hot,22,0.027938
...,...,...,...,...
365,AgglomerativeClustering,One-Hot,87,0.027938
366,AgglomerativeClustering,One-Hot,88,0.027938
367,AgglomerativeClustering,One-Hot,89,0.027938
368,AgglomerativeClustering,One-Hot,90,0.027938


In [282]:
k = params['K (nb clusters)'].values[0]
model = AgglomerativeClustering().fit(X)
labels = model.labels_
df["kmeans"] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(kmeans.labels_.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["kmeans"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['kmeans'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

### Sentence transformers embedding

In [287]:
algo = 'AgglomerativeClustering'
embed = 'Sentence transformers'


# Normalize the embeddings to unit length
X = embeddings_st /  np.linalg.norm(embeddings_st, axis=1, keepdims=True)

# Perform kmean clustering
gmm = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
gmm.fit(X)

labels = gmm.labels_

score  =  silhouette_score(X, labels)
results.loc[((results['algorithme'] == algo) & \
            (results['embedding'] == embed)), 'Score Silhouette'] = score


# for k in clusters:
#     gaac = AgglomerativeClustering().fit(X)
#     labels = gaac.labels_

#     score  =  silhouette_score(X, labels)
#     results.loc[((results['algorithme'] == algo) & \
#                 (results['K (nb clusters)'] == k) & \
#                 (results['embedding'] == embed)), 'Score Silhouette'] = score

results
    

,algorithme,embedding,K (nb clusters),Score Silhouette
0,K-Means,One-Hot,18,0.439495
1,K-Means,One-Hot,19,0.462772
2,K-Means,One-Hot,20,0.480734
3,K-Means,One-Hot,21,0.499245
4,K-Means,One-Hot,22,0.511111
...,...,...,...,...
439,AgglomerativeClustering,Sentence transformers,87,0.237712
440,AgglomerativeClustering,Sentence transformers,88,0.237712
441,AgglomerativeClustering,Sentence transformers,89,0.237712
442,AgglomerativeClustering,Sentence transformers,90,0.237712


*Stock resulting clusters into a CSV file*

In [288]:
tab = results[((results['algorithme'] == algo) & \
                  (results['embedding'] == embed))].drop(columns=['K (nb clusters)']).drop_duplicates()
                  
tab

,algorithme,embedding,Score Silhouette
370,AgglomerativeClustering,Sentence transformers,0.237712


In [290]:
df['clusters'] = labels

# Pour mieux interpréter, on assigne un label significatif à nos clusters
# On retient le terme pour chaque cluster dont la valeur TF + DF est la plus élevée
current_labels = set(labels.tolist())

desired_labels = {x : None for x in current_labels} # (on initialise à None)

for label in current_labels:
    cluster = df[df["clusters"] == label]
    max_freq = cluster['TF + DF'].max()
    new_label = cluster[cluster['TF + DF'] == max_freq]['Terme'].values[0]

    desired_labels[label] = new_label

df['Cluster'] = df['clusters'].map(desired_labels)

df.sort_values(["Cluster"], 
        axis=0,
        ascending=[False], 
        inplace=True)


# On stocke les résultats dans un CSV
base_path = '../06-clustering/'
file_path = base_path + algo + '_' + embed + '.csv'
df[['Cluster', 'Terme', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)', 'TF + DF']].sort_values(['Cluster', 'Fréquence totale (TF)', 'Fréquence documentaire totale (DF)'],
            ascending = [True, False, False]).to_csv(file_path)

df = df.drop(columns=['Cluster'])

In [291]:
base_path = '../06-clustering/'
file_path = base_path + 'results_clustering.csv'
results.to_csv(file_path)